## TRATAMENTO DE DADOS JIRA 

Versão 1.0

Autor: Filipe Amalfi

## Importanto as bibliotecas

In [2]:
import pandas as pd 
import os
from datetime import datetime


## Importação dos arquivos

In [5]:
df = pd.read_csv( 'Jira 02_2024.csv', sep = ',')

## Padronizando das colunas

Removendo os prefixos Campo personalizado, Link de item interno e Link de item externo das colunas


In [6]:
df.columns = df.columns.str.replace('Campo personalizado \(', '', regex=True)
df.columns = df.columns.str.replace('\)', '', regex=True) 
df.columns = df.columns.str.replace('Link de item interno \(', '', regex=True)
df.columns = df.columns.str.replace('\)', '', regex=True)
df.columns = df.columns.str.replace('Link de item externo \(', '', regex=True)
df.columns = df.columns.str.replace('\)', '', regex=True)

## Tratamento de datas

Criando a váriavel "meses" para usar como padrão para as datas que precisam ser tratadas

In [7]:
meses = {
    'jan': 'Jan',
    'fev': 'Feb',
    'mar': 'Mar',
    'abr': 'Apr',
    'mai': 'May',
    'jun': 'Jun',
    'jul': 'Jul',
    'ago': 'Aug',
    'set': 'Sep',
    'out': 'Oct',
    'nov': 'Nov',
    'dez': 'Dec'
}

Substitui os nomes abreviados dos meses pelas versões em inglês nas colunas 'Resolvido' e 'Criado'.
Converte as colunas 'Criado' e 'Resolvido' para o formato de data e hora.
Extrai o mês da coluna 'Criado' e armazena em 'Ref'.
Extrai a hora da coluna 'Criado' e 'Resolvido' e as armazena em 'Hora de criação' e 'Hora de resolução', respectivamente.
Converte as colunas 'Criado' e 'Resolvido' para conter apenas a data, removendo informações de hora.

In [8]:
df['Resolvido'] = df['Resolvido'].replace(meses, regex=True)
df['Criado'] = df['Criado'].replace(meses, regex=True)
df['Criado'] = pd.to_datetime(df['Criado'], format='%d/%b/%y %I:%M %p')
df['Ref'] = df['Criado'].dt.month
df['Hora de criação'] = df['Criado'].dt.time
df['Criado'] = df['Criado'].apply(lambda x: x.date())
df['Resolvido'] = pd.to_datetime(df['Resolvido'], format='%d/%b/%y %I:%M %p')
df['Hora de resolução'] = df['Resolvido'].dt.time
df['Resolvido'] = df['Resolvido'].apply(lambda x: x.date())

Preenche colunas vazias 'Data Prevista de Termino' e 'Data de termino real' com base em outras colunas.
Substitui nomes abreviados de meses por suas versões em inglês nas colunas.
Converte as colunas para o formato de data e hora apropriado.

In [9]:
df['Data Prevista de Termino'] = df.apply(lambda row: row['Data prevista de término'] if pd.isna(row['Data Prevista de Término']) else row['Data Prevista de Término'], axis=1)
df['Data de termino real'] = df.apply(lambda row: row['Data de término real.1'] if pd.isna(row['Data de término real']) else row['Data de término real'], axis=1)
df['Data de termino real'] = df['Data de termino real'].replace(meses, regex=True)
df['Data Prevista de Termino'] = df['Data Prevista de Termino'].replace(meses, regex=True)
df['Data Prevista de Termino'] = pd.to_datetime(df['Data Prevista de Termino'], format='%d/%b/%y %I:%M %p')
df['Data de termino real'] =  pd.to_datetime(df['Data de termino real'], format='%d/%b/%y %I:%M %p')

A Diferença em dias entre as colunas 'Data Prevista de Termino' e 'Data de termino real' e armazena o resultado na coluna 'Dias Gastos'.

Em resumo, ele determina quantos dias se passaram entre a data prevista de término e a data de término real de um evento e registra esse valor na coluna 'Dias Gastos'.

In [10]:
df['Dias Gastos'] = (df['Data Prevista de Termino'] - df['Data de termino real']).dt.days


## Dropando das colunas

Essas colunas específicas serão excluídas do DataFrame, resultando em um DataFrame com apenas as colunas desejadas.

In [11]:
df = df.drop(columns = ['Data Prevista de Término','Data de término real', 'Data de término real.1', 'Data prevista de término'])

## Exportando o arquivo em CSV

In [12]:
df.to_csv('Jira_202402_tratado.csv',index = False)